# Szyfrowanie blokowe

## Tryby kodowania

Ponieważ wiadomości, jakie chcemy zakodować, są zwykle znacznie większe od rozmiaru bloku, musimy używać jakiegoś trybu kodowania. Najbardziej naiwne podzielenie wiadomości na bloki odpowiednich rozmiarów i zakodowanie osobno każdego z nich (ECB) nie zapewnia nam bezpieczeństwa.

### ECB

Tryb elektronicznej książki kodowej (ang. Electronic CodeBook – ECB) – jeden z najprostszych trybów szyfrowania wiadomości z wykorzystaniem szyfru blokowego. W trybie tym blok tekstu jawnego jest szyfrowany w blok szyfrogramu. Możliwe jest niezależne szyfrowanie oraz deszyfrowanie bloków wiadomości, takie zachowanie pozwala, teoretycznie, stworzyć książkę kodową tekstu jawnego i odpowiadającemu mu szyfrogramu, która będzie zawierała 2n różnych wpisów (n – długość bloku w bitach).

Wadą tego trybu jest fakt, że kryptoanalitycy, dysponując kilkoma tekstami jawnymi i odpowiadającymi im szyfrogramami, mogą rozpocząć odtwarzanie książki kodowej – dla szyfrów z długimi kluczami całkowite odtworzenie książki kodowej jest jednak nierealne. Atakujący ma także możliwość zmiany wiadomości bez znajomości klucza.

### CBC

Tryb wiązania bloków zaszyfrowanych (z ang. Cipher Block Chaining – CBC) – jeden z trybów pracy szyfrów blokowych wykorzystujący sprzężenie zwrotne, samosynchronizujący się; w trybie tym blok tekstu jawnego jest sumowany modulo 2 z szyfrogramem poprzedzającego go bloku w związku z czym wynik szyfrowania jest zależny od poprzednich bloków. Pierwszy blok, przed zaszyfrowaniem, jest sumowany modulo dwa z losowo wygenerowanym wektorem początkowym IV (ang. initialization vector), wektor ten nie musi być utrzymywany w tajemnicy.

### CFB

Tryb sprzężenia zwrotnego szyfrogramu (z ang. Cipher Feedback – CFB) – jeden z trybów działania szyfrów blokowych, przeznaczony do szyfrowania strumieni danych. Szyfrowanie nie może być jednak rozpoczęte zanim nie zostanie odebrany pełny blok danych do zaszyfrowania.

Szyfr blokowy działający w trybie sprzężenia zwrotnego szyfrogramu działa na rejestrze, który jest w stanie pomieścić pełny blok danych przeznaczonych do szyfrowania. Przed rozpoczęciem procedury szyfrowania rejestr ten wypełniany jest losowymi danymi, które umownie nazwane są wektorem początkowym (ang. IV - initialization vector). Zawartość tego rejestru jest szyfrowana a następnie n-skrajnych, lewych bitów jest sumowana modulo dwa z n pierwszymi bitami tekstu jawnego – w ten sposób powstaje pierwsze n-bitów szyfrogramu. Zaszyfrowane w ten sposób bity zapisywane są na n-skrajnych, prawych bitach kolejki, jednocześnie pozostałe bity kolejki przesuwane są w lewo i procedura szyfrowania jest powtarzana[1].

Liczba n jest zależna od trybu CFB – możliwe jest szyfrowanie bit po bicie (1-bitowy CFB), bajt po bajcie (8-bitowy CFB) lub dowolne inne.

-------------
Źródło: [Wikipedia](https://pl.wikipedia.org/wiki/Szyfr_blokowy)

## Prygotowanie

### Wczytanie bibliotek

In [24]:
from Crypto.Cipher import DES,AES
from Crypto.Random import get_random_bytes
from typing import Iterable
from itertools import product
import math
import hashlib
import os
from IPython.display import clear_output

### Przydatne funkcje

In [2]:
def padd_data(data : bytes, block_size : int) -> bytes:
    padding = ord('@')
    diff = block_size - len(data) % block_size
    
    return data + bytes([padding] * diff)

def strToCodes(string : str) -> Iterable[int]:
    return (ord(c) for c in string)

def codesToStr(codes : Iterable[int]) -> str:
    return ''.join((chr(c) for c in codes))

In [54]:
def display_progres(procentage_done : float, bar_length = 20) -> None:
    bar_length = 20
    fill_length = round(bar_length * procentage_done)
    
    bar = '[{}{}]'.format('#' * fill_length, '-' * (bar_length - fill_length))
    display = bar + f' {procentage_done:.0%}'
    print(display)
    
    
display_progres(0.4)

[########------------] 40%


In [3]:
from PIL import Image
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
 

def expand_data(data):
    return data + b"\x00"*(16-len(data)%16) 
 

def convert_to_RGB(data):
    pixels = []
    counter = 2

    for i in range(len(data)-1):
        if counter == 2:
            r = int(data[i])
            g = int(data[i+1])
            b = int(data[i+2])

            pixels.append((r,g,b))
            counter = 0
        else:
            counter += 1
    return pixels


def encrypt(input_filename,mode,key):
 
    img_in = Image.open(input_filename)
    data = img_in.convert("RGB").tobytes() 
 
    data_expanded = expand_data(data)

    if mode == "CBC":
        iv = get_random_bytes(16)
        aes = AES.new(key, AES.MODE_CBC, iv)
    elif mode == "ECB":
        aes = AES.new(key, AES.MODE_ECB)
       
    encrypted_data = convert_to_RGB(aes.encrypt(data_expanded)[:len(data)])
    
    img_out = Image.new(img_in.mode, img_in.size)
    img_out.putdata(encrypted_data)
    
    name = ''.join(input_filename.split('.')[:-1])
    img_format = str(input_filename.split('.')[-1])

    output_filename = name + '_' + mode + '_encrypted.' + img_format

    img_out.save(output_filename, img_format)

### Szyfr DES

In [4]:
key = 'key12345'
data = 'secret12'
iv = get_random_bytes(8)
print(data)

des = DES.new(key, DES.MODE_CBC, iv)
cryptogram = des.encrypt(data)
print(cryptogram)

des = DES.new(key, DES.MODE_CBC, iv)
restored = des.decrypt(cryptogram)
print(restored)

### Szyfr AES

In [5]:
key = 'key4567890123456'
data = 'Ala ma dwa koty.'
iv = get_random_bytes(16)

print(data)

des = AES.new(key, DES.MODE_CBC, iv)
cryptogram = des.encrypt(data)
print(cryptogram)

des = AES.new(key, DES.MODE_CBC, iv)
restored = des.decrypt(cryptogram)
print(restored)

## Zadania z iSoda

Zadania z iSoda

### Zadanie 1

Napisz algorytm obliczający entropię.  
Implementacja na podstawie [artykułu](http://nfsec.pl/hakin9/entropy.pdf).

In [6]:
def entropy(data : bytes) -> float:
    count = {i : 0 for i in range(256)}
    for b in data: count[b] += 1
    
    p = lambda b: count[b] / len(data)
    entropy = sum((p(b) * count[b] for b in range(256)))
        
    return 1 - entropy / len(data)

def iSod_entropy(data : bytes) -> float:
    count = {i : 0 for i in range(256)}
    for b in data: count[b] += 1
    p = lambda b: count[b] / len(data)
    
    entropy = 0
    for b in range(256):
        prob = p(b)
        if prob > 0:
            entropy += prob * math.log2(prob)
#     entropy = -sum((p(b) * math.log2(p(b)) for b in range(256)))
    return -entropy
    

### Zadanie 2

Porównaj entropie tekstu naturalnego z entropia kryptogramu.

In [7]:
poem = None
with open('Dziewczyna.txt') as file:
    poem = file.read()
    
e = entropy(poem.encode())
print(f'Entropia wiersza (pdf) = {e}')

e = iSod_entropy(poem.encode())
print(f'Entropia wiersza (iSod) = {e}')
print()

key = 'key12345'
iv = get_random_bytes(8)
data = padd_data(poem.encode(), 8)

des = DES.new(key, DES.MODE_CBC, iv)
cryptogram = des.encrypt(data)

e = entropy(cryptogram)
print(f'Entropia kryptogramu DES = {e}')

key = 'key4567890123456'
iv = get_random_bytes(16)
data = padd_data(poem.encode(), 16)

des = AES.new(key, DES.MODE_CBC, iv)
cryptogram = des.encrypt(data)

e = entropy(cryptogram)
print(f'Entropia kryptogramu AES = {e}')


Entropia wiersza (pdf) = 0.952523347795931
Entropia wiersza (iSod) = 4.9664311123485

Entropia kryptogramu DES = 0.9957518951591002
Entropia kryptogramu AES = 0.9957142168209877


Wygląda na to, że entropia liczona metodą z iSoda nie jest noramlzowana.

### Zadanie 3

Porównaj wynik szyfrowania w trybach ECB i CBC.  
Jaka jest entropia kryptogramów?

In [8]:
poem
with open('Dziewczyna.txt') as file:
    poem = file.read()
    
print('Syfr DES')
key = 'key12345'
data = padd_data(poem.encode(), 8)

des = DES.new(key, DES.MODE_ECB)
e = entropy(des.encrypt(data))
print(f'Entropia w trybie ECB: {e}')

iv = get_random_bytes(8)
des = DES.new(key, DES.MODE_CBC, iv)
e = entropy(des.encrypt(data))
print(f'Entropia w trybie CBC: {e}')

# des_cbc = DES.new(key, DES.MODE_CBC)
print('\nSyfr AES')
key = 'key4567890123456'
data = padd_data(poem.encode(), 16)

aes = AES.new(key, AES.MODE_ECB)
e = entropy(aes.encrypt(data))
print(f'Entropia w trybie ECB: {e}')

iv = get_random_bytes(16)
aes = AES.new(key, AES.MODE_CBC, iv)
e = entropy(aes.encrypt(data))
print(f'Entropia w trybie CBC: {e}')


Syfr DES
Entropia w trybie ECB: 0.9957271630418758
Entropia w trybie CBC: 0.9957363769679006

Syfr AES
Entropia w trybie ECB: 0.9956987847222222
Entropia w trybie CBC: 0.9957573784722222


Teraz to samo, ale na obrazie

In [9]:
file_name = 'demo24.bmp'
key = 'key4567890123456'

encrypted_CBC = encrypt(file_name, 'CBC', key)
encrypted_ECB = encrypt(file_name, 'ECB', key)

file_name1 = 'demo24_CBC_encrypted.bmp'

with open(file_name, 'rb') as file:
    data = file.read()
    print(f'Entropia orginału {iSod_entropy(data)}')

with open(file_name1, 'rb') as file:
    data = file.read()
    print(f'Entropia w trybie CBC {iSod_entropy(data)}')
    
file_name2 = 'demo24_ECB_encrypted.bmp'

with open(file_name2, 'rb') as file:
    data = file.read()
    print(f'Entropia w trybie EBC {iSod_entropy(data)}')
    
    

# print(f'Entropia w trybie CBC {iSod_entropy(encrypted_CBC)}')
# print(f'Entropia w trybie ECB {iSod_entropy(encrypted_ECB)}')
# print(encrypted_CBC)

Entropia orginału 5.051968777777436
Entropia w trybie CBC 7.999024459472189
Entropia w trybie EBC 7.705863173996502


### Zadanie 4

Napisz program szyfrujący pliki przy pomocy algorytmu AES w trybie CBC.

In [10]:
from pathlib import Path

def encryptFile(file_name : str, key : str) -> None:
    data = None
    with open(file_name, 'rb') as file: data = file.read()
    data = padd_data(data, 16)
    
    aes = AES.new(key, AES.MODE_CBC, iv)
    encrypted = aes.encrypt(data)
        
    new_file = Path(file_name).with_suffix('.aes')
    with open(new_file, 'wb') as output: output.write(encrypted)
        
encryptFile('Dziewczyna.txt', 'key4567890123456')

### Zadanie 5

Zaproponuj algorytm tworzenia klucza na podstawie hasła podawanego przez człowieka.

Implementacja bazująca na algorytmie **KDF1**

```
INPUT:
Z, shared secret, a byte string;
Hash, hash function with output hLen bytes;
kLen, intended length of keying material in bytes;
[OtherInfo], optional extra shared material.
OUTPUT: Derived key, K, of length kLen bytes.

Set d = ceiling(kLen/hLen).
Set T = "", the empty string.
for Counter = 0 to d-1 do:
    C = IntegerToString(Counter, 4)
    T = T || Hash(Z || C || [OtherInfo])
Output the first kLen bytes of T as K.
```

---------------
[Źródło](https://web.archive.org/web/20101229081854/http://di-mgt.com.au/cryptoKDFs.html)

In [11]:
def create_key_KDF1(secret : bytes, hash_name : str, desired_length : int, salt : bytes = b'') -> bytes:
    h = hashlib.new(hash_name)
    d = math.ceil(desired_length / h.block_size)
    T = b''
    
    for i in range(d):
        c = i.to_bytes(4, byteorder='big')
        h = hashlib.new(hash_name)
        h.update(secret + c + salt)
        T += h.digest()
        
    return T[:desired_length]
        

create_key_KDF1(b'aa', 'sha256', 8, b'02')

b'\xac\xbfK\\\xbd\xe9\xca\xa4'

Autorska implementacja

Wielokrotne hashowanie poprzedniego wyniku i soli

In [12]:
def create_key_simple(secret : bytes, desired_length : int, iterations : int, salt : bytes = b'') -> bytes:
    if desired_length > hashlib.sha256().block_size:
        raise ValueError(f'This function can create keys of max {hashlib.sha256().block_size} size. Recived {desired_length} lenght')
    
    result = secret
    for i in range(iterations):
        h = hashlib.sha256()
        h.update(result)
        h.update(salt)
        result = h.digest()
        
    return result[:desired_length]


create_key_simple(b'aa', 8, 1000, b'02')    

b'\xc1h\xb3w\x06\x99!\xb1'

Wykorzystanie funkcji z biblioteki

In [13]:
hashlib.pbkdf2_hmac('sha256', b'password', b'salt', 100000, 8)

b'\x03\x94\xa2\xed\xe32\xc9\xa1'

### Zadanie 6

Określ ile znaków [a-z] należy podać, żeby entropia hasła zbliżyła się do 256-bitowego klucza AES.

Na początek postaram się określić jaka jest entropia 256-bitowego klucza AES. Ponieważ w prawdopodobieństwo wystąpienia każdego znaku jest jednakowe mogę skożystać ze wzoru przedstawionego poniżej.

\begin{equation}
H = k * \log_2{n}
\end{equation}

* **H** - entropia
* **k** - długość hasła
* **n** - moc alfabetu

Ponieważ klucz jest 256-btowy składa się z 32 bajtów. Czyli **k** = 32  
Każdy bajt może mieć 256 wartości. Czyli **n** = 256  

Obliczam więc entropię klucza AES.

\begin{equation}
H = 32 * \log_2{256} = 256
\end{equation}

Dla zanków [a-z] znana jest moc alfabetu. Pozostaje więc wyznaczyć długosć hasła tak by entropia była równa entropii klucza AES czyli wyniosła **256**.

\begin{equation}
|[a-z]| = 26
\end{equation}

\begin{equation}
H = k * log_2{26} \Leftrightarrow{} H = 256 \\
\Downarrow{} \\
k = 256 / log_2{26} \approx 54.4630
\end{equation}

Wynika z tego, że przy podaniu około **54** znaków z przedziału [a-z] entropia takiego ciągu będzie porównywalna do entropi 256-bitowego klucza AES.

In [14]:
ord('a') - ord('z')
randomBytes = os.urandom(32)
iSod_entropy(b'abcdefghi')

3.169925001442312

### Zadanie 7

Napisz program do ataku brutalnej siły na kryptogram przy wykorzystaniu entropii jako uniwersalnego kryterium zakończenia algorytmu.

In [15]:
def bruteforceDES(cryptogram : bytes, keyAlphabet = range(256)) -> None:
    entropyThreshold = 0.965

    possibleKeys = product(keyAlphabet, repeat=8)
    for k in possibleKeys:
        k = codesToStr(k)
        
        des = DES.new(k, DES.MODE_ECB)
        recovered = des.decrypt(cryptogram)
        
        e = entropy(recovered)
        if e < entropyThreshold:
            print('\nZnaleziono rozwiązanie!\n')
            print(f'Klucz: {k}')
            print(f'Wiadomość: {recovered}')
            return

    print(f'Nie znaleziono rozwiązania')
        


In [16]:
key = 'abcabdbb'
data = padd_data(poem.encode(), 8)

des = DES.new(key, DES.MODE_ECB)
cryptogram = des.encrypt(data)

keyAlphabet = list(range(ord('a'), ord('e') + 1))
# bruteforceDES(cryptogram, keyAlphabet)

#### Dekryptowanie obrazu

`iv = 'aaaaaaaaaaaaaaaa'`  
Metoda: **CBC**  
klucz powstał w sposób `key = PBKDF2(pass, b'abc')`  
Hasł zkłada się z **3** znaków z zakresu [a-z]

In [17]:
from Crypto.Protocol.KDF import PBKDF2

In [55]:
def bruteforce_AES_CBC(cryptogram : bytes, iv : bytes, key_lenght : int = 3 ,keyAlphabet = list(range(256))) -> None:
#     entropyThreshold = 0.965
    entropyThreshold = 7.7    
    
    possibilities_count = len(keyAlphabet) ** key_lenght

    possiblePasswords = product(keyAlphabet, repeat=key_lenght)
    for i, password in enumerate(possiblePasswords):
        plain_text_password = codesToStr(password)
        salt = b'abc'
        key = PBKDF2(password, salt)
        
        des = AES.new(key, AES.MODE_CBC, iv)
        recovered = des.decrypt(cryptogram)
#         e = entropy(recovered)
        e = iSod_entropy(recovered)
        
        clear_output(True)
        display_progres((i+1) / possibilities_count)
        print(f'Hasło: {plain_text_password}      entropia: {e}')
        
        if e < entropyThreshold:
            clear_output()
            print('\nZnaleziono rozwiązanie!\n')
            print(f'Hasło: {plain_text_password}')
#             print(f'Wiadomość: {recovered}')
            
            rgb = convert_to_RGB(recovered)
            img_out = Image.new(img_in.mode, img_in.size)
            return plain_text_password

    print(f'Nie znaleziono rozwiązania')
        


In [57]:
file_name = 'we800_CBC_encrypted.bmp'

img_in = Image.open(file_name)
data = img_in.convert("RGB").tobytes() 

alphabet = list(range(ord('a'), ord('f') + 1))
iv = b'a' * 16

password = bruteforce_AES_CBC(data, iv, 3, alphabet)



Znaleziono rozwiązanie!

Hasło: fed


Odkodowanie obrazu

In [33]:
key = PBKDF2(password, b'abc')
recovered = AES.new(key, AES.MODE_CBC, iv).decrypt(data)
recovered = convert_to_RGB(recovered)
# print(recovered[:20])

img_out = Image.new(img_in.mode, img_in.size)
img_out.putdata(recovered)
img_out.save('Decrypted image.bmp', 'bmp')

# password
